# 1.- Creación de la sesión Spark (1 punto)
# • Crea una sesión de Spark con el nombre "AnalisisCarreras".

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

# 1. Crea una sesión de Spark con el nombre "Analisis Carreras"
spark = SparkSession.builder \
    .appName("Analisis Carreras") \
    .getOrCreate()

# 2.- Carga de datos en un DataFrame (1 punto)
# • Carga los datos desde el archivo carreras.json en un DataFrame de Spark.
# • Muestra los primeros registros

In [12]:
df = spark.read.option("multiline", "true").json("carreras.json")
df.show()

+--------------------+--------------------+---+---------+----------------+
|                area|             carrera| id|inscritos|     universidad|
+--------------------+--------------------+---+---------+----------------+
|Económicas y Admi...|Ingeniería Comercial|  1|     3200|     U. de Chile|
|   Ciencias Sociales|             Derecho|  2|     2900|             PUC|
|               Salud|            Medicina|  3|     2500|U. de Concepción|
|   Ciencias Sociales|          Psicología|  4|     1800|  U. de Santiago|
|Ingeniería y Tecn...|    Ingeniería Civil|  5|     3100|           UTFSM|
+--------------------+--------------------+---+---------+----------------+



# 3.- Exploración del DataFrame (1 punto)
# • Imprime el esquema del DataFrame para visualizar los tipos de datos de cada columna

In [13]:
# 3. Imprime el esquema del DataFrame para visualizar los tipos de datos de cada columna
df.printSchema()

root
 |-- area: string (nullable = true)
 |-- carrera: string (nullable = true)
 |-- id: long (nullable = true)
 |-- inscritos: long (nullable = true)
 |-- universidad: string (nullable = true)



# 4.- Consultas con Spark SQL (3 puntos)
# Realiza las siguientes consultas usando Spark SQL:
# a) Obtener todas las carreras con más de 2500 inscritos.
# b) Contar cuántas carreras pertenecen a cada área de conocimiento.
# c) Mostrar las universidades que ofrecen más de una carrera en la lista de datos.

In [14]:
# Crea una vista temporal para poder ejecutar consultas SQL
df.createOrReplaceTempView("carreras")

# a) Obtener todas las carreras con más de 2500 inscritos [cite: 24]
query_a = spark.sql("SELECT * FROM carreras WHERE inscritos > 2500")
query_a.show()

# b) Contar cuántas carreras pertenecen a cada área de conocimiento [cite: 25]
query_b = spark.sql("SELECT area, COUNT(carrera) as numero_carreras FROM carreras GROUP BY area")
query_b.show()

# c) Mostrar las universidades que ofrecen más de una carrera en la lista de datos [cite: 26]
# Note: Based on the provided data, no university has more than one career.
query_c = spark.sql("SELECT universidad, COUNT(carrera) as numero_carreras FROM carreras GROUP BY universidad HAVING COUNT(carrera) > 1")
query_c.show()

+--------------------+--------------------+---+---------+-----------+
|                area|             carrera| id|inscritos|universidad|
+--------------------+--------------------+---+---------+-----------+
|Económicas y Admi...|Ingeniería Comercial|  1|     3200|U. de Chile|
|   Ciencias Sociales|             Derecho|  2|     2900|        PUC|
|Ingeniería y Tecn...|    Ingeniería Civil|  5|     3100|      UTFSM|
+--------------------+--------------------+---+---------+-----------+

+--------------------+---------------+
|                area|numero_carreras|
+--------------------+---------------+
|               Salud|              1|
|Ingeniería y Tecn...|              1|
|Económicas y Admi...|              1|
|   Ciencias Sociales|              2|
+--------------------+---------------+

+-----------+---------------+
|universidad|numero_carreras|
+-----------+---------------+
+-----------+---------------+



# 5.- Creación de una Función Definida por el Usuario (UDF) (2 puntos)
# • Crea una UDF que clasifique las carreras según la cantidad de inscritos:
# o "Alta demanda" si tiene más de 3000 inscritos.
# o "Media demanda" si tiene entre 2000 y 3000 inscritos.
# o "Baja demanda" si tiene menos de 2000 inscritos.
# • Aplica la UDF al DataFrame y agrega la nueva columna "demanda".
# • Muestra el DataFrame actualizado.

In [15]:
# 5. a) Crea una UDF que clasifique las carreras según la cantidad de inscritos [cite: 28]
def clasificar_demanda(inscritos):
    if inscritos > 3000:
        return "Alta demanda"
    elif 2000 <= inscritos <= 3000:
        return "Media demanda"
    else:
        return "Baja demanda"

# Registra la función como una UDF
clasificar_demanda_udf = udf(clasificar_demanda, StringType())

# 5. b) Aplica la UDF al DataFrame y agrega la nueva columna "demanda"
df_carreras_con_demanda = df.withColumn("demanda", clasificar_demanda_udf(df["inscritos"]))

# Muestra el DataFrame actualizado [cite: 33]
df_carreras_con_demanda.show()

+--------------------+--------------------+---+---------+----------------+-------------+
|                area|             carrera| id|inscritos|     universidad|      demanda|
+--------------------+--------------------+---+---------+----------------+-------------+
|Económicas y Admi...|Ingeniería Comercial|  1|     3200|     U. de Chile| Alta demanda|
|   Ciencias Sociales|             Derecho|  2|     2900|             PUC|Media demanda|
|               Salud|            Medicina|  3|     2500|U. de Concepción|Media demanda|
|   Ciencias Sociales|          Psicología|  4|     1800|  U. de Santiago| Baja demanda|
|Ingeniería y Tecn...|    Ingeniería Civil|  5|     3100|           UTFSM| Alta demanda|
+--------------------+--------------------+---+---------+----------------+-------------+



# 6.- Guardado de datos en Parquet (2 punto)
# • Guarda el DataFrame transformado en un archivo Parquet llamado
# "carreras_procesadas.parquet".

In [16]:
# 6. Guarda el DataFrame transformado en un archivo Parquet
df_carreras_con_demanda.write.mode("overwrite").parquet("carreras_procesadas.parquet")

print("DataFrame guardado exitosamente como 'carreras_procesadas.parquet'")

# Opcional: Para verificar el guardado, puedes leer el archivo y mostrar el contenido
df_parquet = spark.read.parquet("carreras_procesadas.parquet")
df_parquet.show()

# Detiene la sesión Spark
spark.stop()

DataFrame guardado exitosamente como 'carreras_procesadas.parquet'
+--------------------+--------------------+---+---------+----------------+-------------+
|                area|             carrera| id|inscritos|     universidad|      demanda|
+--------------------+--------------------+---+---------+----------------+-------------+
|Económicas y Admi...|Ingeniería Comercial|  1|     3200|     U. de Chile| Alta demanda|
|   Ciencias Sociales|             Derecho|  2|     2900|             PUC|Media demanda|
|               Salud|            Medicina|  3|     2500|U. de Concepción|Media demanda|
|   Ciencias Sociales|          Psicología|  4|     1800|  U. de Santiago| Baja demanda|
|Ingeniería y Tecn...|    Ingeniería Civil|  5|     3100|           UTFSM| Alta demanda|
+--------------------+--------------------+---+---------+----------------+-------------+

